## Data Provenance 

1. Where is the data from?
    - The data is from Google's Dynamic World (https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1)
    
2. Is it reliable? (partially subjective)
    - Yes, one of the most up-to-date data on landcover with the highest resolution with data cleaning and pre-processing already done
    
## Time measures

1. Time range of the dataset
    - 2015-06-23T00:00:00Z - 2022-06-24T00:00:00
    
2. Granularity (Weekly, Monthly, Aperiodic, Daily, Timestamp)
    - Frequency of update is 2-5 days depending on the lat, long
    
## Location

1. Is it available for the UK, Brazil/India?
   - Yes

2. Granularity (is it a pin point location?, Rounded coordinates?...)
   - 10m resolution
   
3. Format (shape files, lat long, geographical code)
    - Satellite imagery encoded with lat, long data
    
## Data Gaps and size

1. Size of the data
    - Not sure of exact size but it is lot of data to be downloaded entirely, we need to subset by space and time
    
2. Available columns and columns of interest
    - The dataset provides following labels
    
        Name	              Min	 Max	Description
        
        
        water	               0	 1	    Estimated probability of complete coverage by water
        

        trees	               0	 1	    Estimated probability of complete coverage by trees
        

        grass	               0	 1	    Estimated probability of complete coverage by grass
        
    
        flooded_vegetation	   0	 1	    Estimated probability of complete coverage by flooded vegetation
        

        crops	               0	 1	    Estimated probability of complete coverage by crops

        shrub_and_scrub	       0	 1	    Estimated probability of complete coverage by shrub and scrub

        built	               0	 1	    Estimated probability of complete coverage by built

        bare	               0	 1	    Estimated probability of complete coverage by bare

        snow_and_ice	       0	 1	    Estimated probability of complete coverage by snow and ice

        label	               0	 8	    Index of the band with the highest estimated probability
        
3. Missing data (amount and years/locations missing)
    - N/A

4. Nulls or Nans
    - N/A

5. Primary key
   - Lat/Long/Time
   
6. Columns to join on
   - Lat/Long/Time
   
7. Any additional fields
    - N/A
    
## Ease of access

1. Is there a free to use license?
    - Yes

2. How is data accessed? (API, download)
    - Through Earth Engine library (javascript and phython support)
    
3. Is the documentation straightforward?
    - Javascript examples are rich, python not so much
    
4. Any limits on how much data we can download?
    - No


In [1]:
import ee
import geemap

In [2]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
# Set the region of interest by simply drawing a polygon on the map
region = Map.user_roi
if region is None:
    region = ee.Geometry.BBox(-43.40678683735475, -22.977088536079737, -43.4040048161822, -22.984520667299844, )

Map.centerObject(region)

In [4]:
# Set the date range
start_date = '2021-01-01'
end_date = '2022-01-01'

In [12]:
# Create a Sentinel-2 image composite
image = geemap.dynamic_world_s2(region, start_date, end_date)
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
Map.addLayer(image, vis_params, 'Sentinel-2 image')

In [6]:
# Create Dynamic World land cover composite
landcover = geemap.dynamic_world(region, start_date, end_date, return_type='hillshade')
Map.addLayer(landcover, {}, 'Land Cover')

In [7]:
# Add legend to the map
Map.add_legend(title="Dynamic World Land Cover", builtin_legend='Dynamic_World')
Map

Map(bottom=148524.0, center=[-22.91792293614603, -43.06640625000001], controls=(WidgetControl(options=['positi…

In [11]:
landcover.getInfo()

{'type': 'Image',
 'bands': [{'id': 'vis-red',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 1},
   'crs': 'EPSG:3857',
   'crs_transform': [10, 0, 0, 0, 10, 0]},
  {'id': 'vis-green',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 1},
   'crs': 'EPSG:3857',
   'crs_transform': [10, 0, 0, 0, 10, 0]},
  {'id': 'vis-blue',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 1},
   'crs': 'EPSG:3857',
   'crs_transform': [10, 0, 0, 0, 10, 0]}]}

In [64]:
t = image.select('B1')

In [105]:
g = geemap.ee_to_numpy(image, 'B1', region, default_value=2)

In [106]:
g

array([[[None, None]]], dtype=object)

In [9]:
region_point = ee.Geometry.Point(-43.19608028049068, -22.954356561829943)

In [10]:
region_point

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Point",
    "arguments": {
      "coordinates": {
        "constantValue": [
          -43.19608028049068,
          -22.954356561829943
        ]
      }
    }
  }
})

In [11]:
image_new = geemap.dynamic_world_s2(region_point, start_date, end_date)

In [12]:
image_new

In [13]:
probabilityBands = [
  'water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub',
  'built', 'bare', 'snow_and_ice'
];

In [14]:
dwTimeSeries = image.select(probabilityBands);

In [15]:
dwTimeSeries

In [18]:
scale = 1000  # scale in meters

In [28]:
t = image_new.sample(region_point, scale).first().get('B1').getInfo()

In [29]:
t

1553.5